# 08 Metal

* 빛을 반사하는 금속 재질을 구현합니다.

In [1]:
from typing import List
import sys
from tqdm import tqdm

import numpy as np
import numpy.linalg as LA

import matplotlib.pyplot as plt

infinity = sys.float_info.max

In [2]:
def normalize(vec: np.array, eps: float = 1e-6):
    return vec / (LA.norm(vec) + eps)


def blend(color1: np.array, color2: np.array, t: float):
    return (1.0 - t) * color1 + t * color2


def linear_to_gamma(linear_component: np.array) -> np.array:
    return np.where(linear_component > 0, np.sqrt(linear_component), linear_component)


def scale_color(color: np.array, num_sample_per_pixel: int) -> np.array:
    # divide the color by the number of samples.
    scale = 1.0 / num_sample_per_pixel
    rgb = scale * color
    rgb = np.clip(rgb, 0.0, 0.999)
    return rgb


def random_in_unit_sphere() -> np.array:
    p = np.random.uniform(low=-1.0, high=1.0, size=[3])
    return normalize(p)


def random_in_hemisphere(normal: np.array) -> np.array:
    p = random_in_unit_sphere()
    return p if np.dot(p, normal) > 0.0 else -p


def near_zero(vec: np.array, eps: float = 1e-6) -> bool:
    return np.all(np.abs(vec) < eps)


def reflect(vec: np.array, normal: np.array) -> np.array:
    return vec - 2 * np.dot(vec, normal) * normal

In [3]:
class Ray:
    def __init__(self, origin: np.array, direction: np.array) -> None:
        self.origin: np.array = origin
        self.direction: np.array = normalize(direction)

    def at(self, t: float) -> np.array:
        return self.origin + t * self.direction

In [4]:
class Material:
    def scatter(self, r_in: Ray, record: "HitRecord") -> tuple[bool, np.array, Ray]:
        return False, None, None

In [5]:
class Lambertian(Material):
    def __init__(self, albedo: np.array):
        self.albedo = albedo

    def scatter(self, r_in: Ray, record: "HitRecord") -> tuple[bool, np.array, Ray]:
        scatter_direction = record.normal + random_in_unit_sphere()

        if near_zero(scatter_direction):
            scatter_direction = record.normal

        scattered = Ray(origin=record.point, direction=scatter_direction)
        return True, self.albedo, scattered

In [6]:
class Metal(Material):
    def __init__(self, albedo: np.array):
        self.albedo = albedo

    def scatter(self, r_in: Ray, record: "HitRecord") -> tuple[bool, np.array, Ray]:
        reflected = reflect(r_in.direction, record.normal)
        scattered = Ray(origin=record.point, direction=reflected)
        return True, self.albedo, scattered

In [7]:
class HitRecord:
    def __init__(self, point: np.array, normal: np.array, t: float) -> None:
        self.point = point
        self.normal = normal
        self.material: Material = None
        self.t = t
        self.is_front_face = True

    def set_face_normal(self, ray: Ray, outward_normal: np.array) -> None:
        self.is_front_face = np.dot(ray.direction, outward_normal) < 0
        self.normal = outward_normal if self.is_front_face else -outward_normal

In [8]:
class Hittable:
    def hit(self, ray: Ray, t_min: float, t_max: float) -> HitRecord:
        raise NotImplementedError

In [9]:
class Sphere(Hittable):
    def __init__(self, center: np.array, radius: float, material: Material) -> None:
        self.center = center
        self.radius = radius
        self.material = material

    def hit(self, ray: Ray, t_min: float, t_max: float) -> bool:
        dir_center_to_origin = ray.origin - self.center

        a = np.dot(ray.direction, ray.direction)
        half_b = np.dot(dir_center_to_origin, ray.direction)
        c = np.dot(dir_center_to_origin, dir_center_to_origin) - self.radius**2.0

        discriminant = half_b**2 - a * c
        if discriminant < 0:
            return None
        sqrt_d = np.sqrt(discriminant)

        # find the nearest root that lies in the acceptable range.
        t = (-half_b - sqrt_d) / a
        if t < t_min or t_max < t:
            t = (-half_b + sqrt_d) / a
            if t < t_min or t_max < t:
                return None

        point = ray.at(t)
        outward_normal = (point - self.center) / self.radius
        hit_record = HitRecord(point=point, normal=outward_normal, t=t)
        hit_record.set_face_normal(ray=ray, outward_normal=outward_normal)
        hit_record.material = self.material

        return hit_record

In [10]:
class HittableList(Hittable):
    def __init__(self) -> None:
        self.objects: List[Hittable] = []

    def clear(self) -> None:
        self.objects.clear()

    def add(self, object: Hittable) -> None:
        self.objects.append(object)

    def hit(self, ray: Ray, t_min: float, t_max: float) -> HitRecord:
        record = None
        closest_so_far = t_max

        for object in self.objects:
            tmp_record = object.hit(ray=ray, t_min=t_min, t_max=closest_so_far)
            if tmp_record is not None:
                closest_so_far = tmp_record.t
                record = tmp_record

        return record

In [11]:
class Camera:
    def __init__(self, image_width: int, image_height: int) -> None:
        self.aspect_ratio = 16.0 / 9.0
        self.viewport_height = 2.0
        self.viewport_width = self.aspect_ratio * self.viewport_height
        self.focal_length = 1.0

        self.origin = np.array([0.0, 0.0, 0.0])
        self.horizontal_vec = np.array([self.viewport_width, 0.0, 0.0])
        self.vertical_vec = np.array([0.0, self.viewport_height, 0.0])
        self.frontal_vec = np.array([0.0, 0.0, self.focal_length])
        self.lower_left_corner = (
            self.origin
            - self.horizontal_vec / 2.0
            - self.vertical_vec / 2.0
            - self.frontal_vec
        )

        self.pixel_delta_u = self.horizontal_vec / image_width
        self.pixel_delta_v = self.vertical_vec / image_height

    def get_ray(self, u: float, v: float) -> Ray:
        return Ray(
            origin=self.origin,
            direction=(
                self.lower_left_corner + u * self.pixel_delta_u + v * self.pixel_delta_v
            )
            - self.origin,
        )

In [12]:
def ray_color(ray: Ray, world: Hittable, depth: int) -> np.array:
    if depth <= 0:
        color = np.zeros(3)
        return color

    record = world.hit(ray=ray, t_min=0.0, t_max=infinity)
    if record is not None:
        scatter_info = record.material.scatter(ray, record)
        if scatter_info[0]:
            return scatter_info[1] * ray_color(ray=scatter_info[2], world=world, depth=depth - 1)
        return np.zeros(3)

    t = 0.5 * (ray.direction[1] + 1.0)
    color1 = np.array([1.0, 1.0, 1.0])
    color2 = np.array([0.5, 0.7, 1.0])
    return blend(color1=color1, color2=color2, t=t)

In [13]:
# image
aspect_ratio = 16.0 / 9.0
image_width = 100
image_height = int(image_width / aspect_ratio)
num_sample_per_pixel = 100
max_depth = 5

image = np.zeros([image_height, image_width, 3])

In [14]:
# material
material_ground = Lambertian(np.array([0.8, 0.8, 0.0]))
material_center = Lambertian(np.array([0.1, 0.2, 0.5]))
material_left = Metal(np.array([0.8, 0.8, 0.8]))
material_right = Metal(np.array([0.8, 0.6, 0.2]))

# world
world = HittableList()
world.add(Sphere(center=np.array([0.0, -100.5, -1.0]), radius=100.0, material=material_ground))
world.add(Sphere(center=np.array([0.0, 0.0, -1.2]), radius=0.5, material=material_center))
world.add(Sphere(center=np.array([-1.0, 0.0, -1.0]), radius=0.5, material=material_left))
world.add(Sphere(center=np.array([ 1.0, 0.0, -1.0]), radius=0.5, material=material_right))

In [15]:
# camera
camera = Camera(image_width, image_height)

In [ ]:
%%time

for row in tqdm(range(image_height)):
  for col in range(image_width):
    for _ in range(num_sample_per_pixel):
      u = col + np.random.uniform(low=0.0, high=1.0)
      v = row + np.random.uniform(low=0.0, high=1.0)
      ray = camera.get_ray(u, v)
      image[row, col] += ray_color(ray=ray, world=world, depth=max_depth)

image = scale_color(image, num_sample_per_pixel=num_sample_per_pixel)
image = linear_to_gamma(image)

In [ ]:
plt.imshow(image, origin="lower")
plt.show()

In [18]:
class Metal(Material):
    def __init__(self, albedo: np.array, fuzz: float):
        self.albedo = albedo
        self.fuzz = fuzz if fuzz < 1 else 1

    def scatter(self, r_in: Ray, record: "HitRecord") -> tuple[bool, np.array]:
        reflected = reflect(r_in.direction, record.normal)
        reflected = normalize(reflected) + (self.fuzz * random_in_unit_sphere())

        scattered = Ray(origin=record.point, direction=reflected)
        if np.dot(scattered.direction, record.normal) > 0:
            return True, self.albedo, scattered
        else:
            return False, self.albedo, scattered

In [19]:
# material
material_ground = Lambertian(np.array([0.8, 0.8, 0.0]))
material_center = Lambertian(np.array([0.1, 0.2, 0.5]))
material_left = Metal(np.array([0.8, 0.8, 0.8]), 0.3)
material_right = Metal(np.array([0.8, 0.6, 0.2]), 1.0)

# world
world = HittableList()
world.add(Sphere(center=np.array([0.0, -100.5, -1.0]), radius=100.0, material=material_ground))
world.add(Sphere(center=np.array([0.0, 0.0, -1.2]), radius=0.5, material=material_center))
world.add(Sphere(center=np.array([-1.0, 0.0, -1.0]), radius=0.5, material=material_left))
world.add(Sphere(center=np.array([ 1.0, 0.0, -1.0]), radius=0.5, material=material_right))

In [ ]:
%%time

for row in tqdm(range(image_height)):
  for col in range(image_width):
    for _ in range(num_sample_per_pixel):
      u = col + np.random.uniform(low=0.0, high=1.0)
      v = row + np.random.uniform(low=0.0, high=1.0)
      ray = camera.get_ray(u, v)
      image[row, col] += ray_color(ray=ray, world=world, depth=max_depth)

image = scale_color(image, num_sample_per_pixel=num_sample_per_pixel)
image = linear_to_gamma(image)

In [ ]:
plt.imshow(image, origin="lower")
plt.show()